# ML для оценки параметров экономических моделей **(кейс Центробанка)**

Импортирование необходимых библиотек для работы с нейронной сетью 

In [2]:
import numpy as np

# Загрузка набора данных

Загрузка и распределение, по переменным, необходимых данных из датасета

In [3]:
clip = np.load('dataset/clip.npy')
w = np.load('dataset/weight.npy')
x_train = np.load("dataset/y_smp_test.npy")
y_train = np.load("dataset/pars_smp_train.npy")

# Импортирование библиотек для работы с нейронной сетью

In [34]:
from keras.models import Sequential
from keras.layers import Dense
from keras.src.layers import GRU, Dropout

# Создание модели для прогнозирования макроэкономических параметров

In [21]:
model = Sequential()
model.add(GRU(units=128,
              return_sequences=True,
              input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(GRU(units=64))
model.add(Dense(32))
# model.add(Dropout(0.5))
model.add(Dense(15))
model.summary()
model.compile(optimizer='adam',
              loss='mse', metrics=["mae"])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 200, 128)          51072     
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 15)                495       
                                                                 
Total params: 90895 (355.06 KB)
Trainable params: 90895 (355.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Обучение модели 

In [22]:
model.compile(optimizer='adam',
              loss='mse', metrics=["mae"])

model.fit(x_train[:100000,:,], y_train[:100000,:,], epochs=5, batch_size=128)

Epoch 1/5
8/8 [==============================] - 8s 659ms/step - loss: 0.8503 - mae: 0.6664
Epoch 2/5
8/8 [==============================] - 6s 708ms/step - loss: 0.3515 - mae: 0.3852
Epoch 3/5
8/8 [==============================] - 6s 703ms/step - loss: 0.2315 - mae: 0.2975
Epoch 4/5
8/8 [==============================] - 6s 770ms/step - loss: 0.1790 - mae: 0.2682
Epoch 5/5
8/8 [==============================] - 6s 807ms/step - loss: 0.1400 - mae: 0.2368


# Использование сети для предсказания

In [28]:
pred = model.predict(x_train[:100001:200001])

1/1 [==============================] - 0s 455ms/step


# Создание и обучение моделей для рассчёта квантиль

In [60]:
def create_models(x_train, y_train, s):
    model = Sequential()
    model.add(GRU(units=128,
                  return_sequences=True,
                  input_shape=(15, 1)))
    # model.add(GRU(units=64))
    model.add(Dense(32))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.summary()
    model.compile(optimizer='adam',
                  loss='mse', metrics=["mae"])
    model.fit(x_train,
              y_train[:, :, s],
              epochs=5, batch_size=128, validation_split=0.1)
    return model

In [46]:
def merge_arrays(arr1, arr2, arr3, arr4, arr5, arr6):
    fin_arr = np.empty((arr1.shape[0],arr1.shape[1],6))
    for i in range(len(arr1)):
        for j in range(len(arr1[0])):
            semi_arr = np.hstack([arr1[i][j], arr2[i][j],
                                      arr3[i][j], arr4[i][j],
                                      arr5[i][j], arr6[i][j]])
            fin_arr[i][j] = semi_arr
    return fin_arr

In [61]:
arr = []

x_train = np.load('dataset/pars_smp_test.npy')
y_train = np.load('dataset/random_submit.npy')

for i in range(6):
        model = create_models(x_train, y_train, i)
        pred = model.predict(pred)
        print(pred.shape)
        arr.append(pred[:100000])
        
fin_arr = merge_arrays(arr[0], arr[1],arr[2],arr[3],arr[4],arr[5])

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_40 (GRU)                (None, 15, 128)           50304     
                                                                 
 dense_84 (Dense)            (None, 15, 32)            4128      
                                                                 
 dropout_36 (Dropout)        (None, 15, 32)            0         
                                                                 
 dense_85 (Dense)            (None, 15, 1)             33        
                                                                 
Total params: 54465 (212.75 KB)
Trainable params: 54465 (212.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
704/704 [==============================] - 13s 16ms/step - loss: 1.0005 - mae: 0.7981 - val_loss: 1.0009 - val_mae: 0.7981
Epoch 2/5
704/

# Оценка точности работы 

In [62]:
def calculate_metrics(forecast, true_values, weights = 1/15*np.ones(shape=(15,11)), clip = 10**6*np.ones(shape=(15,6)), quantiles = [0.1,0.25,0.5,0.75,0.9]):
    # Define diff between forecast
    diff = true_values - forecast
    # Mean loss
    RMSE = np.mean(np.minimum(diff[:,:,0]**2, clip[None,:,0]), axis=0)**0.5
    L_mean = np.exp(np.sum(-weights[:,0]*RMSE))
    # Quantile loss
    quantile_loss = np.mean(np.minimum(((diff[:,:,1:]>0)*(np.array(quantiles)[None,None,:]) + (diff[:,:,1:]<=0)*(1 - np.array(quantiles)[None,None,:]))*abs(diff[:,:,1:]), clip[None,:,1:]), axis=0)
    L_quantile = np.exp(np.sum(-weights[:,1:6]*quantile_loss))
    # Calibration loss
    L_calibration = np.exp(np.sum(-weights[:,6:]*np.abs(np.array(quantiles)-np.mean(diff[:,:,1:]<0, axis=0))))
    return 0.45*L_mean + 0.45*L_quantile + 0.1*L_calibration

In [63]:
metrics = calculate_metrics(fin_arr, x_train, weights= w, clip= clip, quantiles = [0.1,0.25,0.5,0.75,0.9])
with open('metrics.npy', 'wb') as f:
    np.save(f, metrics)

0.33620312926087054